<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/food12_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data download from google drive

food data
- label : 12
- label_names = 'apple_pie', 'cheesecake', 'pizza', 'waffles', 'ramen', 'bibimbap', 'fried_rice', 'club_sandwich', 'hamburger', 'hot_dog', 'sashimi', 'sushi'
- resized : 224 * 224 * 3
- dataset : train 700, val 200, test 100

In [4]:
!gdown 1--zzEz-AUajDmmXsVeBMVOks3aJ6yeXJ

Downloading...
From: https://drive.google.com/uc?id=1--zzEz-AUajDmmXsVeBMVOks3aJ6yeXJ
To: /content/food12_dataset.zip
100% 285M/285M [00:00<00:00, 327MB/s]


In [3]:
# gdown 불가시 설치
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [5]:
!unzip -q food12_dataset.zip

# test

In [5]:
!pip install split-folders
# !pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# .kaggle 폴더 생성
!mkdir -p ~/.kaggle/
# kaggle.josn .kaggle로 복사
!cp kaggle.json ~/.kaggle/
# 해당 폴더 목록 확인(복사 잘 되었는지 확인)
!ls ~/.kaggle

kaggle.json


In [3]:
cd ~

/root


In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
cd /content

/content


In [6]:
!kaggle datasets download -d kmader/food41

100% 5.30G/5.30G [05:49<00:00, 17.5MB/s]
100% 5.30G/5.30G [05:49<00:00, 16.3MB/s]


In [7]:
!unzip -q food41.zip

In [6]:
import splitfolders

import os

import cv2
import numpy as np
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

import pickle

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [7]:
label_names = ['apple_pie', 'cheesecake', 'pizza', 'waffles', 'ramen', 'bibimbap', 'fried_rice', 'club_sandwich', 'hamburger', 'hot_dog', 'sashimi', 'sushi']
len(label_names)

12

In [8]:
folder_path = '/content'

In [9]:
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in label_names:
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'apple_pie': ['/content/apple_pie/3503020.jpg',
  '/content/apple_pie/3214508.jpg',
  '/content/apple_pie/1088809.jpg',
  '/content/apple_pie/1595134.jpg',
  '/content/apple_pie/3549466.jpg',
  '/content/apple_pie/3203950.jpg',
  '/content/apple_pie/1332030.jpg',
  '/content/apple_pie/874809.jpg',
  '/content/apple_pie/2725375.jpg',
  '/content/apple_pie/3267436.jpg',
  '/content/apple_pie/693210.jpg',
  '/content/apple_pie/64846.jpg',
  '/content/apple_pie/937118.jpg',
  '/content/apple_pie/1753935.jpg',
  '/content/apple_pie/1456028.jpg',
  '/content/apple_pie/2881877.jpg',
  '/content/apple_pie/2235547.jpg',
  '/content/apple_pie/1725573.jpg',
  '/content/apple_pie/2402925.jpg',
  '/content/apple_pie/2373323.jpg',
  '/content/apple_pie/224785.jpg',
  '/content/apple_pie/2601590.jpg',
  '/content/apple_pie/946403.jpg',
  '/content/apple_pie/3241750.jpg',
  '/content/apple_pie/2586042.jpg',
  '/content/apple_pie/2745241.jpg',
  '/content/apple_pie/2067073.jpg',
  '/content/apple_pie/

In [10]:
len(dataset)

12

In [14]:
!mkdir resized

In [15]:
cd /content/resized

/content/resized


In [16]:
for label in label_names:
    os.mkdir(label) 

In [17]:
# resize 덮어씌우기
for label, filenames in tqdm(dataset.items()):
    for filename in filenames:
        img = cv2.imread(filename)

        # 이미지의 x, y가 128이 넘을 경우 작게해주기
        percent = 1
        if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세보다 크면 가로를 128으로 맞추고 세로를 비율에 맞춰서
            percent = 224/img.shape[1]
        else :
            percent = 224/img.shape[0]

        img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                # 이미지 범위 지정
        y,x,h,w = (0,0,img.shape[0], img.shape[1])

        # 그림 주변에 검은색으로 칠하기
        w_x = (224-(w-x))/2  # w_x = (128 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
        h_y = (224-(h-y))/2

        if(w_x < 0):         # 크기가 -면 0으로 지정.
            w_x = 0
        elif(h_y < 0):
            h_y = 0

        M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
       
        cv2.imwrite('/content/resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)

  0%|          | 0/12 [00:00<?, ?it/s]

In [28]:
import os

def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [21]:
label_names

['apple_pie',
 'cheesecake',
 'pizza',
 'waffles',
 'ramen',
 'bibimbap',
 'fried_rice',
 'club_sandwich',
 'hamburger',
 'hot_dog',
 'sashimi',
 'sushi']

In [32]:
print(get_files_count('/content/dataset/val/apple_pie'))
print(get_files_count('/content/dataset/val/cheesecake'))
print(get_files_count('/content/dataset/val/pizza'))
print(get_files_count('/content/dataset/val/waffles'))
print(get_files_count('/content/dataset/val/ramen'))
print(get_files_count('/content/dataset/val/bibimbap'))
print(get_files_count('/content/dataset/val/fried_rice'))
print(get_files_count('/content/dataset/val/club_sandwich'))
print(get_files_count('/content/dataset/val/hamburger'))
print(get_files_count('/content/dataset/val/hot_dog'))
print(get_files_count('/content/dataset/val/sashimi'))
print(get_files_count('/content/dataset/val/sushi'))

200
200
200
200
200
200
200
200
200
200
200
200


In [18]:
splitfolders.ratio('/content', output='/content/dataset', seed=77, ratio=(0.7, 0.2, 0.1))

Copying files: 12009 files [00:01, 6172.26 files/s]


In [36]:
folder_path = '/content/food12_224' # 각자 원하는 드라이브 경로에 맞게 수정합시다.

In [ ]:
with open(folder_path+'food12_224*224.pickle', 'wb') as f:
    pickle.dump(, f)

In [19]:
import zipfile
import os

In [40]:
os.getcwd()

'/content'

In [23]:
cd /content

/content


In [24]:
import shutil
shutil.make_archive('food12_dataset', 'zip', '/content/dataset')

'/content/food12_dataset.zip'

In [25]:
!gdown 1--zzEz-AUajDmmXsVeBMVOks3aJ6yeXJ

Downloading...
From: https://drive.google.com/uc?id=1--zzEz-AUajDmmXsVeBMVOks3aJ6yeXJ
To: /content/food12_dataset.zip
100% 285M/285M [00:00<00:00, 290MB/s]


In [47]:
pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [48]:
!gdown 1ejSNtqYeTV52oco4Pp4523mSUFtZoGZh

Downloading...
From: https://drive.google.com/uc?id=1ejSNtqYeTV52oco4Pp4523mSUFtZoGZh
To: /content/food12.zip
100% 285M/285M [00:00<00:00, 370MB/s]
